In [1]:
import tiktoken, json
import semantic_kernel as sk
from Plugins.google_search import GoogleSearchPlugin
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

class SKReport:
    def __init__(self, report_title:str):
        self.service_id = "default"
        self.api_key, self.org_id = sk.openai_settings_from_dot_env()

        self.report_title=report_title
        print(self.report_title) ##===||========================>

        # Create the SK Kernel
        self.kernel = sk.Kernel()
        # Add the OpenAI service
        self.kernel.add_service(OpenAIChatCompletion(service_id=self.service_id, ai_model_id="gpt-3.5-turbo-0125", 
                                                     api_key=self.api_key, 
                                                     org_id=self.org_id,
                                                     )
                                )
        # Add the plugins
        self.plugin = self.kernel.import_plugin_from_prompt_directory("./Plugins/", "ReportPlugin")
        self.outlineFunc = self.plugin["OutlineJSON"] ## Outline Plugin
        self.introFunc = self.plugin["Introduction"] ## Introduction Plugin
        # self.quesFunc = self.plugin["Questioning"] ## Questioning Plugin

        # Outline Generator
        # self.outlineSplitter()

        # Introduction
        # Introduction = self.kernel.invoke(self.introFunc, sk.KernelArguments(introduction=outline_dict["introduction"],
        #                                                                     reportLayout = str(OutlineJSON),
        #                                                                     )
        #                                     )

    async def outlineSplitter(self):
        generatedOutline = await self.kernel.invoke(self.outlineFunc, sk.KernelArguments(input=self.report_title))
        data = json.loads(str(generatedOutline).replace("'", '"'))
        self.dictOutline = {
            "title" : data['report']['title'],
            "introduction" : data['report']['introduction'],
            "body" : data['report']['body'],
            "conclusion" : data['report']['conclusion'],
            "recommendations" : data['report']['recommendations']
        }
        self.title = self.dictOutline["title"]
        self.introduction = self.dictOutline["introduction"]
        self.body = self.dictOutline["body"]
        self.conclusion = self.dictOutline["conclusion"]
        self.recommendations = self.dictOutline["recommendations"]

        return self.dictOutline

    def token_count(self, string):
        encoding = tiktoken.get_encoding("cl100k_base")
        """Returns the number of tokens in a text string."""
        num_tokens = len(encoding.encode(string))
        return num_tokens

    async def columnist(self, heading, num_results:int=5):
        requestion = await self.kernel.invoke(self.quesFunc, 
                                         sk.KernelArguments(keywords=heading, main_topic = self.title),
                                         )
        # Googling
        GGSearch = GoogleSearchPlugin()
        search_results = GGSearch.search_google(requestion, num_results=num_results)
        # Getting the content
        urls = list(set([s for s in search_results]))
        for url in urls:
            content = GGSearch.get_content(urls)


In [2]:
report_title = "Báo cáo thị trường năng lượng thế giới năm 2023"
OGReport = SKReport(report_title)
await OGReport.outlineSplitter()
OGReport.dictOutline

Báo cáo thị trường năng lượng thế giới năm 2023


KeyError: 'Function Questioning not found.'

In [ ]:
str(OGReport.generatedOutline.get_inner_content().choices[0].message.content)